# Algoritmo para ejecutar CVA

CVA (Credit Valuation Adjustment) es una técnica que permite calcular la diferencia entre el valor de un portofolio dado y el valor de un portafolio de renta fija, considerando las probabilidades de incumplimiento para cada unidad de tiempo, con el fin de encontrar el valor de mercado de los derivados incluyendo los riesgos de incumplimiento.

Un método que puede explotar las capacidades de Pyhton es el diseño de simulaciones para obtener la exposición futura esperada (EFE) a partir de simulaciones para cada unidad de tiempo. Se propone que la dinámica entre el precio del subyacente en cada unidad de tiempo respecto la anterior se modele a través de un movimiento browniano geométrico.

La fórmula que caracteriza la valuación es: 

$$CVA(T)\,=\,\int_{0}^{T} E^{Q} \left [ LGD \frac{B_{0}}{B_{t}} E(t) | t\,=\,\tau \right ] dPD(0,t)$$

Donde $T$ es la duración del derivado, $B_{t}$ es el valor de una unidad monetaria dada la tasa actual llevada al plazo $t$, $E(t)$ es la exposición al tiempo $t$, $LGD$ es la pérdida esperada dado incumplimiento  y $PD(0,t)$ es la probabilidad que se incumpla en los pagos en alguno de los primeros $t$ periodos de tiempo.

$LGD$ se puede entender en términos simplistas como el porcentaje de deuda que pudiera cubrirse con la venta de activos al tiempo $t$. 

Se puede usar tanto para valuación de derivados como para encontrar el valor de mercado de un portafolio respecto a uno libre de riesgo pero con probabilidad positiva de incumplimiento en los cupones.

# Programa

Este programa realiza dos tareas: i. Calcula la probabilidad de incumplimiento del instrumento en P$ para posteriormente ii. Calcular el CVA usando pronósticos de FX obtenidos mediante simulaciones de Monte Carlo, para posteriomente calcular el MKM y la EPE (promedio de los MKM positivos por periodo de tiempo) y el percentil 95 (EFE) para cada periodo de tiempo. Asumimos que el contrato tiene un plazo de 90 días y los intrumentos de renta fija que se usaron para obtener la probabilidad de incumplimiento pagan semi-anualmente.

## I. Probabilidad de incumplimiento

In [43]:
import datetime
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from scipy.optimize import Bounds

def inmediato_posterior(d1,d2):
    '''
    Devuelve la fecha inmediatamente después de d1 tomando al número 
    de día de d2.
    '''
    m=d2.month
    d=d2.day
    m6=(m+6)%12
    if m6==0:
        m6=12
    if d1.month < m or (d1.month==m and d>=d1.day):
        resp = datetime.strptime(str(d)+"/"+str(m)+"/"+str(d1.year),"%d/%m/%Y")
    else:
        resp = datetime.strptime(str(d)+"/"+str(m6)+"/"+str(d1.year),"%d/%m/%Y")
    return resp
        

def fecha_6M(d1):
    '''
    Entrega la fecha con el mismo número de día exactamente 6 meses después 
    '''
    if (d1.month+6)%12 == 0:
        m2=12
    else:
        m2=(d1.month+6)%12
    if d1.month+6 >12:
        y2=d1.year+1
    else:
        y2=d1.year
    d2=str(d1.day)+"/"+str(m2)+"/"+str(y2)
    return datetime.strptime(d2,"%d/%m/%Y")

def days360(start_date, end_date, method_eu=False):
    '''
    Replica lo que hace dias360 de Excel.
    '''
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

def plazos_fechas(maturity_date,current_day=datetime.today()):
    '''
    Esta función encuentra los plazos (en días) y las fechas en las 
    que el instrumento paga el cupón, asumiendo que se pagan semi-
    anualmente.
    '''
    
    #Fechas: 
    cd=current_day
    md=datetime.strptime(maturity_date,"%d/%m/%Y")
    aux=inmediato_posterior(cd,md)
    plazos=[days360(cd,aux)]
    fechas=[aux]
    while (md-aux).days!=0:
        aux=fecha_6M(aux)
        plazos.append(days360(cd,aux))
        fechas.append(aux)
    return plazos,fechas

def cupon_fd_fdInter(maturity_date,cupon,YTM,tasa_interbancaria,nominal=100,current_day=datetime.today()):
    '''
    Esta función devuelve el precio de cupón, el factor de descuento
    y el factor de descuento con la tasa de interés interbancaria, con
    el fin de calcular el precio sucio. Se mantiene el supuesto que
    los instrumentos pagan semianualmente.
    '''
    
    plazos,fechas=plazos_fechas(maturity_date)
    cupones,fd,fdInter=[],[],[]
    c=cupon*nominal/2
    
    for i in range(len(plazos)):
        if i<len(plazos)-1:
            cupones.append(c)
        else:
            cupones.append(c+100)
    
    for p,date in zip(plazos,fechas):
        if (current_day-date).days >=0:
            fd.append(0)
        else:
            fd.append(1/(1+YTM/2)**(p/180))
    
    for p,date in zip(plazos,fechas):
        if (current_day-date).days >=0:
            fdInter.append(0)
        else:
            fdInter.append(1/(1+tasa_interbancaria/2)**(p/180))
    return cupones,fd,fdInter

def precio_sucio(cupones,df):
    '''
    Esta función calcula el precio sucio del instrumento.
    Precio sucio es igual al precio limpio más 
    la proporción del cupon a recibir relativo a los días que 
    pasaron desde fecha actual hasta el pago del primer cupon.
    '''
    array_cupon=np.array(cupones)
    array_df=np.array(df)
    return np.sum(array_cupon*array_df)

def func_obj(h,*args):
    '''
    Función objetivo a resolver.
    '''
    cupones=args[0]
    plazos=args[1]
    dfInter=args[2]
    nominal=args[3]
    R=args[4]
    df=args[5]
    return (1/2)*(precio_sucio(cupones,df)-precio_riesgoso(R,nominal,plazos,dfInter,h,cupones))**2

def precio_riesgoso(R,nominal,plazos,dfInter,h,cupones):
    '''
    Esta función calcula el precio del instrumento
    incluyendo la probabilidad por incumplimiento. 
    '''
    s=0
    for i in range(len(plazos)):
        s+=cupones[i]*dfInter[i]*(1-h)**(i+1)+dfInter[i]*R*nominal*h*(1-h)**i
    return s

def proba_incumplimiento(maturity_date,cupon,YTM,tasa_interbancaria,R,nominal=100,current_day=datetime.today()):
    '''
    Esta función calculará la probabilidad de incumplimiento tal que el
    precio ajustado por el riesgo sea igual al precio sucio del instrumento.
    
    Asumimos que el instrumento paga los cupones semianualmente.
    
    Llamará al solver para encontrar el valor de h. 
    
    h: Probabilidad de incumplimiento.
    '''
    
    plazos,fechas=plazos_fechas(maturity_date,current_day=datetime.today())
    cupones,fd,fdInter=cupon_fd_fdInter(maturity_date,cupon,YTM,tasa_interbancaria,nominal=100,current_day=datetime.today())
    x_ini=.05
    cotas=Bounds(lb = 1e-4, ub = 1-1e-4)
    h = minimize(func_obj,x0 = x_ini,method='L-BFGS-B',args=(cupones,plazos,fdInter,nominal,R,fd)) 
    return h

### Cálculo de la probabilidad de incumplimiento

In [42]:
import datetime
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from scipy.optimize import Bounds

maturity_date="13/03/2027"
cupon=6.5/100
YTM=8.5508/100
tasa_interbancaria=0.67/100
R=25/100
nominal=100
current_day=datetime.today()
h=proba_incumplimiento(maturity_date,cupon,YTM,tasa_interbancaria,R,nominal,current_day)
print(f"La probabilidad de incumplimiento para el instrumento es de: {round(100*h.x[0],5)}%")

La probabilidad de incumplimiento para el instrumento es de: 4.51932%


## II. Simulaciones de Monte Carlo para pronosticar el valor del subyacente

In [92]:
import datetime
import numpy as np
import scipy,math
from datetime import datetime
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.stats import norm
from math import log,exp

np.random.seed(54321)

def tasa_USD_continua(tasa_USD_anual,plazo=90):
    '''
    Genera la tasa para los contratos en USD para un tiempo continuo.
    '''
    return (360/plazo)*log(1+tasa_USD_anual*plazo/360)

def tasa_MXNImplicita_continua(S0,ptos_fwd,tasa_USD_anual,plazo=90):
    '''
    Genera la tasa para los contratos en MXN para un tiempo continuo.
    '''
    precio_fwd=S0+ptos_fwd/1e4
    tasa_imp_MXN=((precio_fwd/S0)*(1+tasa_USD_anual*plazo/360)-1)*(360/plazo)
    return (360/plazo)*log(1+tasa_imp_MXN*plazo/360)

def actualiza_paso(x,tasa_MXN_continua,tasa_USD_cont,vol,paso):
    '''
    Actualiza el siguiente paso del método de Monte Carlo.
    '''
    z=norm.rvs(loc=0,scale=1,size=1) #Simula un muestra tamaño 1 de una distribución normal estándar
    return x*exp((tasa_MXN_continua-tasa_USD_cont-0.5*vol**2)*(paso/360)+z*vol*(paso/360)**0.5)

def simulacion_subyacente(S0,ptos_fwd,tasa_USD_anual,vol,plazo=90,paso=1):
    '''
    Esta función devolverá una matriz con cada una de las simulaciones del 
    valor del subyacente para cada unidad de tiempo, hasta que se cumpla el plazo 
    del contrato.
    '''
    n_sim=int(1e3)
    tasa_MXN_continua=tasa_MXNImplicita_continua(S0,ptos_fwd,tasa_USD_anual,plazo=90)
    tasa_USD_cont =tasa_USD_continua(tasa_USD_anual,plazo=90)
    M_sim=[]
    for i in range(n_sim):
        col=[S0]
        for j in range(plazo):
            col.append(actualiza_paso(col[-1],tasa_MXN_continua,tasa_USD_cont,vol,paso)) 
        M_sim.append(col)
    M_sim=np.array(M_sim)
    return M_sim

def actualiza_contrato(x,tasa_MXN_cont,tasa_USD_cont,k,nominal,plazo,precio_fwd):
    '''
    Actualiza el MTM por periodo de tiempo y trayectoria
    '''
    return nominal*(x*exp((tasa_MXN_cont-tasa_USD_cont)*(plazo-k)/360)-precio_fwd)*exp(-tasa_MXN_cont*(plazo-k)/360)

def valuacion_fwd(M,S0,ptos_fwd,tasa_USD_anual,plazo=90,nominal=1e7):
    '''
    Esta función calcula el mark-to-market (MTK) tomando el valor de la tasa FX para cada día del plazo
    y lo trae a valor presente.
    '''
    precio_fwd=S0+ptos_fwd/1e4
    tasa_MXN_continua=tasa_MXNImplicita_continua(S0,ptos_fwd,tasa_USD_anual)
    tasa_USD_cont=tasa_USD_continua(tasa_USD_anual)
    n_sim,plazos=M.shape
    M_contrato=[]
    for i in range(n_sim):
        f=[]
        for j in range(1,plazos):
            f.append(actualiza_contrato(M[i][j],tasa_MXN_continua,tasa_USD_cont,j,nominal,plazo,precio_fwd))
        M_contrato.append(f)
    return M_contrato 

def CVA(vol,S0,ptos_fwd,tasa_USD_anual,h,TIIE=7.13/100,plazo=90,nominal=1e7,paso=1):
    '''
    Esta función calcula la valuación por ajuste de crédito (CVA) de un derivado de FX
    MXN-USD a plazo de 90 días incluyendo la probabilidad de incumplimiento encontrada arriba.
    '''
    EPE=[]
    EFE=[]
    df=[]
    pr_cond=[]
    M=simulacion_subyacente(S0,ptos_fwd,tasa_USD_anual,vol,plazo,paso)
    M_contrato=valuacion_fwd(M,S0,ptos_fwd,tasa_USD_anual,plazo,nominal)
    M_contrato=np.array(M_contrato)
    f,c=M_contrato.shape
    for i in range(c):
        v=M_contrato[:,i]
        EPE.append(v[v>0].mean())
        EFE.append(np.percentile(v,95))
    for i in range(1,plazo+1):
        df.append(1/(1+TIIE*91/360)**(i/91))
        pr_cond.append((1-h)**((i-1)/180)-(1-h)**(i/180))
    pr_cond=np.array(pr_cond)
    df=np.array(df)
    EFE=np.array(EFE)
    EPE=np.array(EPE)
    cva_epe=np.sum(pr_cond*df*EPE)
    cva_efe=np.sum(pr_cond*df*EFE)
    return cva_epe,cva_efe

### Cálculo del CVA

In [95]:
import datetime
import numpy as np
import scipy,math
from datetime import datetime
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.stats import norm
from math import log,exp
np.random.seed(54321)

p=h.x[0] #Probabilidad de incumplimiento del ejercicio anterior
S0=21.77
ptos_fwd=4118
tasa_USD_anual=0.7841/100
vol=20/100
epe,efe=CVA(vol,S0,ptos_fwd,tasa_USD_anual,p,TIIE=7.13/100,plazo=90,nominal=1e7,paso=1)
print("_____________________________________________ CVA __________________________________________________")
print("")
print(f"                                    Mediante EPE: {round(epe,2)}")
print(f"                                    Mediante EFE: {round(efe,2)}")

_____________________________________________ CVA __________________________________________________

                                    Mediante EPE: 275329.38
                                    Mediante EFE: 573765.26
